In [1]:
using Flux, MLDatasets, Statistics
using Flux: onehotbatch, onecold, crossentropy


In [2]:
model = Chain(
    Conv((3, 3), 1=>24, relu),
    x -> maxpool(x, (4,4)),

    x -> reshape(x, :, size(x, 4)),
    Dense(864, 32, relu),
    Dense(32, 10),

    softmax,
)


Chain(
  Conv((3, 3), 1 => 24, relu),          # 240 parameters
  var"#1#3"(),
  var"#2#4"(),
  Dense(864 => 32, relu),               # 27_680 parameters
  Dense(32 => 10),                      # 330 parameters
  NNlib.softmax,
)                   # Total: 6 arrays, 28_250 parameters, 110.953 KiB.

In [3]:
train_set = MNIST(split=:train, dir="./train")[:]
test_set  = MNIST(split=:test, dir="./test")[:]

This program has requested access to the data dependency MNIST.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: THE MNIST DATABASE of handwritten digits
Authors: Yann LeCun, Corinna Cortes, Christopher J.C. Burges
Website: http://yann.lecun.com/exdb/mnist/

[LeCun et al., 1998a]
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner.
    "Gradient-based learning applied to document recognition."
    Proceedings of the IEEE, 86(11):2278-2324, November 1998

The files are available for download at the offical
website linked above. Note that using the data
responsibly and respecting copyright remains your
responsibility. The authors of MNIST aren't really
explicit about any terms of use, so please read the
website to make sure you want to download the
dataset.



Do you want to download the dataset from ["https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz", "https://ossci-datasets.s3.amazonaws.com/mn

(features = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], targets = [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [4]:
using Flux: Data.DataLoader

x_train = Flux.unsqueeze(train_set.features, 3)
x_test = Flux.unsqueeze(test_set.features, 3)

y_train = onehotbatch(train_set.targets, 0:9)
y_test = onehotbatch(test_set.targets, 0:9)

train_data = DataLoader((x_train, y_train); batchsize=128)
test_data = DataLoader((x_test, y_test); batchsize=128)

79-element DataLoader(::Tuple{Array{Float32, 4}, OneHotArrays.OneHotMatrix{UInt32, Vector{UInt32}}}, batchsize=128)
  with first element:
  (28×28×1×128 Array{Float32, 4}, 10×128 OneHotMatrix(::Vector{UInt32}) with eltype Bool,)

In [5]:
loss(x, y) = crossentropy(model(x), y)
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

opt = ADAM(0.0075)

Adam(0.0075, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [8]:
using BenchmarkTools: @btime

number_epochs = 1
@btime for i in 1:number_epochs 
    Flux.train!(loss, Flux.params(model), train_data, opt)
    println("Epoch ", i, ". loss: ", loss(train_data.data[1], train_data.data[2])) 

end

Epoch 1. loss: 0.04488531
Epoch 1. loss: 0.02807118
Epoch 1. loss: 0.025267005
Epoch 1. loss: 0.029351374
  27.863 s (192665 allocations: 26.80 GiB)


In [7]:
acc = Vector{Float64}()

for data in test_data
    push!(acc, accuracy(data[1], data[2]))
        
end

In [8]:
println("Models accuracy:", mean!([1.], acc))

Models accuracy:[0.9811902866242038]
